# **Notebook zur Evaluation - Trajektorien**
***

In [1]:
import csv
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.cm as cm
import plotly.graph_objects as go

***
### **Trajektorien CSVs laden**

In [2]:
def load_csv_to_trajectory(filepath):
    """Lädt die CSV und stellt Trajektorie-Liste wieder her"""
    trajectory = []
    with open(filepath, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Überspringe Kopfzeile

        for row in reader:
            numbers = list(map(float, row))
            trans = numbers[:3]
            rot = numbers[3:]
            trajectory.append((trans, rot))
    return trajectory

In [3]:
hook_num = 1
v = 10
hook_model = 'A'
frame = '_work'
num_interp_points = 101
hole_width = 8
hole_height = 12

In [4]:
traj_optim_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_0' + str(frame) + '.csv'
traj_1_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_1' + str(frame) + '.csv'
traj_2_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_2' + str(frame) + '.csv'
traj_3_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_3' + str(frame) + '.csv'
traj_4_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_4' + str(frame) + '.csv'

In [5]:
trajectory_optim = load_csv_to_trajectory(traj_optim_path)
trajectory_1 = load_csv_to_trajectory(traj_1_path)
trajectory_2 = load_csv_to_trajectory(traj_2_path)
trajectory_3 = load_csv_to_trajectory(traj_3_path)
trajectory_4 = load_csv_to_trajectory(traj_4_path)

In [6]:
print(trajectory_3)

[([71.07486191890325, -507.4553898045242, 333.16322065069517], [-77.74694175050072, 35.42076840596947, 33.47377335430714]), ([65.69129471469898, -500.9853981468624, 338.416850685152], [-77.74694175050072, 35.42076840596947, 33.47377335430714]), ([65.43358267847543, -499.9779561579593, 339.61224667822205], [-77.74694175050072, 35.42076840596947, 33.47377335430714]), ([65.02040266023778, -499.04195016766226, 340.85230675726206], [-77.74694175050072, 35.42076840596947, 33.47377335430714]), ([64.25187165517036, -497.9401598543797, 340.9363798448348], [-77.74694175050072, 35.42076840596947, 33.47377335430714]), ([63.83103300504172, -496.99042981871355, 342.17251148260084], [-77.74694175050072, 35.42076840596947, 33.47377335430714]), ([63.2508188898903, -495.9532495866021, 343.42813381432774], [-77.74694175050072, 35.42076840596947, 33.47377335430714]), ([62.438932824997664, -494.9315263214667, 342.2842746229161], [-77.74694175050072, 35.42076840596947, 33.47377335430714]), ([61.664530733138

***
### **Interpolation der Trajektorie**

In [7]:
def interpolate_trajectory(traj, num_points):
    """
    Interpoliert eine Trajektorie auf `num_points` gleichmäßig verteilte Punkte.
    Nur die translatorischen Komponenten werden verwendet (3D Punkte).
    """
    positions = np.array([p[0] for p in traj])  # Nur Positionen
    rotations = np.array([p[1] for p in traj])  # Nur Rotationen

    # Abstände zwischen aufeinanderfolgenden Punkten
    deltas_trans = np.linalg.norm(np.diff(positions, axis=0), axis=1)
    cumulative_dist_trans = np.concatenate([[0], np.cumsum(deltas_trans)])
    target_distances_trans = np.linspace(0, cumulative_dist_trans[-1], num_points)

    interp_positions = []
    for dist in target_distances_trans:
        idx = np.searchsorted(cumulative_dist_trans, dist) - 1
        idx = np.clip(idx, 0, len(positions) - 2)
        t = (dist - cumulative_dist_trans[idx]) / (cumulative_dist_trans[idx + 1] - cumulative_dist_trans[idx])
        point = (1 - t) * positions[idx] + t * positions[idx + 1]
        interp_positions.append(point)

    # Rotationsteil
    if np.allclose(rotations, rotations[0]):  # Alle Rotationen sind gleich
        interp_rotations = [rotations[0]] * num_points
    else:
        deltas_rot = np.linalg.norm(np.diff(rotations, axis=0), axis=1)
        cumulative_dist_rot = np.concatenate([[0], np.cumsum(deltas_rot)])
        target_distances_rot = np.linspace(0, cumulative_dist_rot[-1], num_points)

        interp_rotations = []
        for rot in target_distances_rot:
            idx = np.searchsorted(cumulative_dist_rot, rot) - 1
            idx = np.clip(idx, 0, len(rotations) - 2)
            denom = cumulative_dist_rot[idx + 1] - cumulative_dist_rot[idx]
            t = 0 if denom == 0 else (rot - cumulative_dist_rot[idx]) / denom
            point = (1 - t) * rotations[idx] + t * rotations[idx + 1]
            interp_rotations.append(point)

    return np.array(interp_positions), np.array(interp_rotations)

**Berechnung der Langloch-Posen im 3D-Raum für alle interpolierten Punkte**

In [8]:
def fix_rpy(rpy):
    roll, pitch, yaw = rpy
    return [roll, -pitch, yaw]  # oder vielleicht yaw auch invertieren – je nach System


def compute_rectangle_corners(trajectory, frame='', width=8, height=12):
    """
    Berechnet die Rechteck-Eckpunkte (4 Stück) für jeden Trajektorienpunkt.

    :param trajectory: Liste von ([x, y, z], [roll, pitch, yaw]) Einträgen
    :param width: Breite des Rechtecks (in X-Richtung)
    :param height: Höhe des Rechtecks (in Y-Richtung)
    :return: Liste von Rechtecken, jedes als Liste von 4 Eckpunkten (jeweils [x, y, z])
    """
    rectangles = []

    # Rechteck im lokalen TCP-Koordinatensystem (XY-Ebene, Zentrum im Ursprung)
    half_w = width / 2
    half_h = height / 2

    if frame == '_work':
        local_corners = np.array([
            [-half_w, half_h, 0],
            [ half_w, half_h, 0],
            [ half_w, -half_h, 0],
            [-half_w, -half_h, 0]
        ])
    else:
        local_corners = np.array([
        [-half_w, 0, -half_h],
        [ half_w, 0, -half_h],
        [ half_w, 0, half_h],
        [-half_w, 0, half_h]
        ])

    for trans, rpy in trajectory:
        # RPY in Radiant, falls nicht schon so
        if frame == '_work':
            rot = R.from_euler('xyz', fix_rpy(rpy), degrees=True)
        else:
            rot = R.from_euler('xyz', rpy, degrees=True)
        rotated_corners = rot.apply(local_corners)
        translated_corners = rotated_corners + np.array(trans)
        rectangles.append(translated_corners.tolist())
    return rectangles

In [9]:
interp_positions_optim, interp_rotations_optim = interpolate_trajectory(trajectory_optim, num_interp_points)
trajectory_optim = list(zip(interp_positions_optim, interp_rotations_optim))
rectangles_optim = compute_rectangle_corners(trajectory_optim, frame=frame, width=hole_width, height=hole_height)

interp_positions_1, interp_rotations_1 = interpolate_trajectory(trajectory_1, num_interp_points)
trajectory_1 = list(zip(interp_positions_1, interp_rotations_1))
rectangles_1 = compute_rectangle_corners(trajectory_1, frame=frame, width=hole_width, height=hole_height)

interp_positions_2, interp_rotations_2 = interpolate_trajectory(trajectory_2, num_interp_points)
trajectory_2 = list(zip(interp_positions_2, interp_rotations_2))
rectangles_2 = compute_rectangle_corners(trajectory_2, frame=frame, width=hole_width, height=hole_height)

interp_positions_3, interp_rotations_3 = interpolate_trajectory(trajectory_3, num_interp_points)
trajectory_3 = list(zip(interp_positions_3, interp_rotations_3))
rectangles_3 = compute_rectangle_corners(trajectory_3, frame=frame, width=hole_width, height=hole_height)

interp_positions_4, interp_rotations_4 = interpolate_trajectory(trajectory_4, num_interp_points)
trajectory_4 = list(zip(interp_positions_4, interp_rotations_4))
rectangles_4 = compute_rectangle_corners(trajectory_4, frame=frame, width=hole_width, height=hole_height)

In [10]:
'''
def trajectory_distance(traj1, traj2, num_points=10):
    """
    Vergleicht zwei Trajektorien anhand von `num_points` gleichverteilten Punkten.
    Gibt die euklidischen Abstände zurück (Liste) und den Mittelwert.
    """
    interp1_trans, interp1_rot = interpolate_trajectory(traj1, num_points)
    interp2_trans, interp2_rot = interpolate_trajectory(traj2, num_points)

    # Punktweise euklidischer Abstand
    distances = np.linalg.norm(interp1_trans - interp2_trans, axis=1)
    rotations = np.linalg.norm(interp1_rot - interp2_rot, axis=1)
    mean_distance = np.mean(distances)
    mean_rotation = np.mean(rotations)
    max_distance = np.max(distances)
    max_rotation = np.max(rotations)
    return distances, mean_distance, max_distance, rotations, mean_rotation, max_rotation
'''

'\ndef trajectory_distance(traj1, traj2, num_points=10):\n    """\n    Vergleicht zwei Trajektorien anhand von `num_points` gleichverteilten Punkten.\n    Gibt die euklidischen Abstände zurück (Liste) und den Mittelwert.\n    """\n    interp1_trans, interp1_rot = interpolate_trajectory(traj1, num_points)\n    interp2_trans, interp2_rot = interpolate_trajectory(traj2, num_points)\n\n    # Punktweise euklidischer Abstand\n    distances = np.linalg.norm(interp1_trans - interp2_trans, axis=1)\n    rotations = np.linalg.norm(interp1_rot - interp2_rot, axis=1)\n    mean_distance = np.mean(distances)\n    mean_rotation = np.mean(rotations)\n    max_distance = np.max(distances)\n    max_rotation = np.max(rotations)\n    return distances, mean_distance, max_distance, rotations, mean_rotation, max_rotation\n'

In [11]:
'''
def plot_trajectory_distances(
    distances=None, 
    mean_distance=None, 
    max_distance=None, 
    rotations=None, 
    mean_rotation=None, 
    max_rotation=None, 
    hook_num=0, 
    trajectory_process=4
):
    """
    Plottet die euklidischen Distanzen und Rotationen entlang der interpolierten Punkte.
    """

    %matplotlib inline
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
    fig.suptitle(f"Vergleich zwischen optimaler und berechneter Trajektorie - Haken {hook_num}, Trajektorie-Ansatz {trajectory_process} – Distanz", fontsize=14)

    # Plot 1: Euklidische Distanzen
    ax1.set_title(f"Euklidsche Distanz translatorisch")
    ax1.plot(range(1, len(distances) + 1), distances, linestyle='-', marker = '.', label='Euklidische Distanz [mm]')

    if mean_distance is not None:
        ax1.plot(range(1, len(distances) + 1), [mean_distance] * len(distances), linestyle='--', color='red', label='Mittelwert')
    
    if max_distance is not None:
        ax1.plot(range(1, len(distances) + 1), [max_distance] * len(distances), linestyle='--', color='magenta', label=f"Max-Wert - {max_distance:.3f}mm")

    ax1.set_ylabel("Distanz [mm]")
    ax1.grid(True)
    ax1.legend()

    
    # Plot 2: Rotationen
    ax2.set_title("Euklidsche Distanz rotatorisch")
    if rotations is not None:
        ax2.plot(range(1, len(rotations) + 1), rotations, linestyle='-', marker = '.', label='Rotationsabweichung [°]', color='orange')

        if mean_rotation is not None:
            ax2.plot(range(1, len(rotations) + 1), [mean_rotation] * len(rotations), linestyle='--', color='red', label='Mittelwert')
        
        if max_rotation is not None:
            ax2.plot(range(1, len(rotations) + 1), [max_rotation] * len(rotations), linestyle='--', color='magenta', label=f"Max-Wert - {max_rotation:.3f}°")

        ax2.set_ylabel("Rotation [°]")
        ax2.grid(True)
        ax2.legend()
    else:
        ax2.text(0.5, 0.5, "Keine Rotationsdaten verfügbar", ha='center', va='center', transform=ax2.transAxes)
        ax2.axis('off')
    
    ax2.set_xlabel("Punktindex (entlang der Trajektorie)")
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Platz für Suptitle
    plt.show()
'''

'\ndef plot_trajectory_distances(\n    distances=None, \n    mean_distance=None, \n    max_distance=None, \n    rotations=None, \n    mean_rotation=None, \n    max_rotation=None, \n    hook_num=0, \n    trajectory_process=4\n):\n    """\n    Plottet die euklidischen Distanzen und Rotationen entlang der interpolierten Punkte.\n    """\n\n    %matplotlib inline\n    \n    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)\n    fig.suptitle(f"Vergleich zwischen optimaler und berechneter Trajektorie - Haken {hook_num}, Trajektorie-Ansatz {trajectory_process} – Distanz", fontsize=14)\n\n    # Plot 1: Euklidische Distanzen\n    ax1.set_title(f"Euklidsche Distanz translatorisch")\n    ax1.plot(range(1, len(distances) + 1), distances, linestyle=\'-\', marker = \'.\', label=\'Euklidische Distanz [mm]\')\n\n    if mean_distance is not None:\n        ax1.plot(range(1, len(distances) + 1), [mean_distance] * len(distances), linestyle=\'--\', color=\'red\', label=\'Mittelwert\')\n   

In [12]:
'''
distances, mean_distance, max_distance, rotations, mean_rotation, max_rotation = trajectory_distance(trajectory_1, trajectory_optim, 100)
plot_trajectory_distances(distances, mean_distance, max_distance, rotations, mean_rotation, max_rotation, hook_num = hook_num, trajectory_process = 4)
'''

'\ndistances, mean_distance, max_distance, rotations, mean_rotation, max_rotation = trajectory_distance(trajectory_1, trajectory_optim, 100)\nplot_trajectory_distances(distances, mean_distance, max_distance, rotations, mean_rotation, max_rotation, hook_num = hook_num, trajectory_process = 4)\n'

***
### **Absolute Distanzen**

In [13]:
'''
def trajectory_difference(traj1, traj2, num_points=10):
    """
    Vergleicht zwei Trajektorien anhand von `num_points` gleichverteilten Punkten.
    Gibt die vorzeichenbehafteten Differenzen für Translation und Rotation zurück.
    Rückgabe:
        diffs_trans: (num_points, 3) - x, y, z
        diffs_rot:   (num_points, 3) - roll, pitch, yaw
    """
    interp1_trans, interp1_rot = interpolate_trajectory(traj1, num_points)
    interp2_trans, interp2_rot = interpolate_trajectory(traj2, num_points)

    # Vorzeichenbehaftete Differenzen
    diffs_trans = interp1_trans - interp2_trans
    diffs_rot = interp1_rot - interp2_rot

    return diffs_trans, diffs_rot
'''

'\ndef trajectory_difference(traj1, traj2, num_points=10):\n    """\n    Vergleicht zwei Trajektorien anhand von `num_points` gleichverteilten Punkten.\n    Gibt die vorzeichenbehafteten Differenzen für Translation und Rotation zurück.\n    Rückgabe:\n        diffs_trans: (num_points, 3) - x, y, z\n        diffs_rot:   (num_points, 3) - roll, pitch, yaw\n    """\n    interp1_trans, interp1_rot = interpolate_trajectory(traj1, num_points)\n    interp2_trans, interp2_rot = interpolate_trajectory(traj2, num_points)\n\n    # Vorzeichenbehaftete Differenzen\n    diffs_trans = interp1_trans - interp2_trans\n    diffs_rot = interp1_rot - interp2_rot\n\n    return diffs_trans, diffs_rot\n'

In [14]:
'''
def plot_signed_trajectory_differences(diffs_trans, hook_num=0, trajectory_process=4):
    """
    Plottet die vorzeichenbehafteten Differenzen (x, y, z) 
    entlang der interpolierten Punkte, inklusive Maximalabweichungslinien.
    """
    num_points = diffs_trans.shape[0]
    indices = range(1, num_points + 1)

    fig, ax1 = plt.subplots(1, 1, figsize=(10, 5))
    fig.suptitle(f"Vorzeichenbehaftete Differenzen – Haken {hook_num}, Trajektorie-Ansatz {trajectory_process}", fontsize=14)

    colors = ["blue", "green", "purple"]
    labels = ["x [mm]", "y [mm]", "z [mm]"]

    for i in range(3):
        values = diffs_trans[:, i]
        ax1.plot(indices, diffs_trans[:, i], label=labels[i], linestyle='-', color=colors[i])

        # Maximaler Betrag mit Vorzeichen
        idx_max = np.argmax(np.abs(values))
        max_val_signed = values[idx_max]
        max_index = indices[idx_max]

        # Linie bei Maximalwert
        ax1.axhline(y=max_val_signed, color=colors[i], linestyle='--', linewidth=1)

        # Annotation (Textfeld)
        ax1.annotate(
            f"{max_val_signed:.2f} mm",
            xy=(max_index, max_val_signed),
            xytext=(max_index + 2, max_val_signed + 0.3),
            textcoords='data',
            fontsize=9,
            color=colors[i],
            arrowprops=dict(arrowstyle='->', color=colors[i]),
            bbox=dict(boxstyle='round,pad=0.3', fc='white', ec=colors[i], lw=1)
        )

    ax1.set_title("Translation: Abweichung in x, y, z [mm]")
    ax1.set_ylabel("Abweichung [mm]")
    ax1.set_xlabel("Punktindex (entlang der Trajektorie)")
    ax1.grid(True)
    ax1.legend()

    plt.tight_layout(rect=[0, 0, 1, 0.92])
    plt.show()
'''

'\ndef plot_signed_trajectory_differences(diffs_trans, hook_num=0, trajectory_process=4):\n    """\n    Plottet die vorzeichenbehafteten Differenzen (x, y, z) \n    entlang der interpolierten Punkte, inklusive Maximalabweichungslinien.\n    """\n    num_points = diffs_trans.shape[0]\n    indices = range(1, num_points + 1)\n\n    fig, ax1 = plt.subplots(1, 1, figsize=(10, 5))\n    fig.suptitle(f"Vorzeichenbehaftete Differenzen – Haken {hook_num}, Trajektorie-Ansatz {trajectory_process}", fontsize=14)\n\n    colors = ["blue", "green", "purple"]\n    labels = ["x [mm]", "y [mm]", "z [mm]"]\n\n    for i in range(3):\n        values = diffs_trans[:, i]\n        ax1.plot(indices, diffs_trans[:, i], label=labels[i], linestyle=\'-\', color=colors[i])\n\n        # Maximaler Betrag mit Vorzeichen\n        idx_max = np.argmax(np.abs(values))\n        max_val_signed = values[idx_max]\n        max_index = indices[idx_max]\n\n        # Linie bei Maximalwert\n        ax1.axhline(y=max_val_signed, col

In [15]:
'''
diffs_trans, diffs_rot = trajectory_difference(trajectory_1, trajectory_optim, 100)
plot_signed_trajectory_differences(diffs_trans, hook_num=2, trajectory_process=1)

diffs_trans, diffs_rot = trajectory_difference(trajectory_2, trajectory_optim, 100)
plot_signed_trajectory_differences(diffs_trans, hook_num=2, trajectory_process=2)

diffs_trans, diffs_rot = trajectory_difference(trajectory_3, trajectory_optim, 100)
plot_signed_trajectory_differences(diffs_trans, hook_num=2, trajectory_process=3)

diffs_trans, diffs_rot = trajectory_difference(trajectory_4, trajectory_optim, 100)
plot_signed_trajectory_differences(diffs_trans, hook_num=2, trajectory_process=4)
'''

'\ndiffs_trans, diffs_rot = trajectory_difference(trajectory_1, trajectory_optim, 100)\nplot_signed_trajectory_differences(diffs_trans, hook_num=2, trajectory_process=1)\n\ndiffs_trans, diffs_rot = trajectory_difference(trajectory_2, trajectory_optim, 100)\nplot_signed_trajectory_differences(diffs_trans, hook_num=2, trajectory_process=2)\n\ndiffs_trans, diffs_rot = trajectory_difference(trajectory_3, trajectory_optim, 100)\nplot_signed_trajectory_differences(diffs_trans, hook_num=2, trajectory_process=3)\n\ndiffs_trans, diffs_rot = trajectory_difference(trajectory_4, trajectory_optim, 100)\nplot_signed_trajectory_differences(diffs_trans, hook_num=2, trajectory_process=4)\n'

***
### **3D-Darstellung**

In [16]:
'''
def fix_rpy(rpy):
    roll, pitch, yaw = rpy
    return [roll, -pitch, yaw]  # oder vielleicht yaw auch invertieren – je nach System


def compute_rectangle_corners(trajectory, frame='', width=8, height=12):
    """
    Berechnet die Rechteck-Eckpunkte (4 Stück) für jeden Trajektorienpunkt.

    :param trajectory: Liste von ([x, y, z], [roll, pitch, yaw]) Einträgen
    :param width: Breite des Rechtecks (in X-Richtung)
    :param height: Höhe des Rechtecks (in Y-Richtung)
    :return: Liste von Rechtecken, jedes als Liste von 4 Eckpunkten (jeweils [x, y, z])
    """
    rectangles = []

    # Rechteck im lokalen TCP-Koordinatensystem (XY-Ebene, Zentrum im Ursprung)
    half_w = width / 2
    half_h = height / 2

    if frame == '_work':
        local_corners = np.array([
            [-half_w, half_h, 0],
            [ half_w, half_h, 0],
            [ half_w, -half_h, 0],
            [-half_w, -half_h, 0]
        ])
    else:
        local_corners = np.array([
        [-half_w, 0, -half_h],
        [ half_w, 0, -half_h],
        [ half_w, 0, half_h],
        [-half_w, 0, half_h]
        ])

    for trans, rpy in trajectory:
        # RPY in Radiant, falls nicht schon so
        if frame == '_work':
            rot = R.from_euler('xyz', fix_rpy(rpy), degrees=True)
        else:
            rot = R.from_euler('xyz', rpy, degrees=True)
        rotated_corners = rot.apply(local_corners)
        translated_corners = rotated_corners + np.array(trans)
        rectangles.append(translated_corners.tolist())
    return rectangles
'''

'\ndef fix_rpy(rpy):\n    roll, pitch, yaw = rpy\n    return [roll, -pitch, yaw]  # oder vielleicht yaw auch invertieren – je nach System\n\n\ndef compute_rectangle_corners(trajectory, frame=\'\', width=8, height=12):\n    """\n    Berechnet die Rechteck-Eckpunkte (4 Stück) für jeden Trajektorienpunkt.\n\n    :param trajectory: Liste von ([x, y, z], [roll, pitch, yaw]) Einträgen\n    :param width: Breite des Rechtecks (in X-Richtung)\n    :param height: Höhe des Rechtecks (in Y-Richtung)\n    :return: Liste von Rechtecken, jedes als Liste von 4 Eckpunkten (jeweils [x, y, z])\n    """\n    rectangles = []\n\n    # Rechteck im lokalen TCP-Koordinatensystem (XY-Ebene, Zentrum im Ursprung)\n    half_w = width / 2\n    half_h = height / 2\n\n    if frame == \'_work\':\n        local_corners = np.array([\n            [-half_w, half_h, 0],\n            [ half_w, half_h, 0],\n            [ half_w, -half_h, 0],\n            [-half_w, -half_h, 0]\n        ])\n    else:\n        local_corners = n

In [17]:
'''
rectangles_optim = compute_rectangle_corners(trajectory_optim, frame = frame)
rectangles_1 = compute_rectangle_corners(trajectory_1, frame = frame)
rectangles_2 = compute_rectangle_corners(trajectory_2, frame = frame)
rectangles_3 = compute_rectangle_corners(trajectory_3, frame = frame)
rectangles_4 = compute_rectangle_corners(trajectory_4, frame = frame)
'''

'\nrectangles_optim = compute_rectangle_corners(trajectory_optim, frame = frame)\nrectangles_1 = compute_rectangle_corners(trajectory_1, frame = frame)\nrectangles_2 = compute_rectangle_corners(trajectory_2, frame = frame)\nrectangles_3 = compute_rectangle_corners(trajectory_3, frame = frame)\nrectangles_4 = compute_rectangle_corners(trajectory_4, frame = frame)\n'

In [18]:
'''
def visualize_rectangles(rectangles, figsize=(8, 8), connect_points=None, 
                         secondary_rectangles=None, secondary_legend_title=None,
                         hook_num = 0, trajectory_process = 1):
    """
    Visualisiert Rechtecke + Trajektorie im 3D-Raum mit Legende und Bezeichnungen der Trajektorienpunkte.
    
    Zusätzlich kann ein zweiter Satz Rechtecke (grau & transparent) dargestellt werden – inkl. Mittelpunktslinie.

    :param rectangles: Liste von Rechtecken (jeweils 4 Punkte [x, y, z])
    :param figsize: Größe der Plotfläche (Breite, Höhe)
    :param connect_points: Optionales Tupel mit zwei Punkten [x, y, z], die verbunden werden sollen
    :param secondary_rectangles: Optionaler zweiter Satz Rechtecke (werden grau dargestellt)
    :param secondary_legend_title: Optionaler Legenden-Eintrag für die grauen Rechtecke
    """
    %matplotlib qt

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, projection='3d')
    fig.suptitle(f"Einfädelung Trajektorie Spitze -> Senke (TCP in WORLD-Frame)", fontsize=14, y=0.98)

    centers = []
    cmap = cm.get_cmap('tab20')
    legend_handles = []

    # Berechnete Trajektorie (farbig)
    for i, rect in enumerate(rectangles):
        color = cmap(i / len(rectangles))
        poly = Poly3DCollection([rect], facecolors=color, alpha=0.2, edgecolors=color)
        ax.add_collection3d(poly)

        if i == 0:
            label = 'Pre-Position'
        elif i == 1:
            label = 'Spitze'
        elif i == len(rectangles) - 1:
            label = 'Loslassen'
        else:
            label = str(i + 1)

        legend_handles.append(plt.Line2D([0], [0], marker='o', color='w', label=label,
                                         markersize=10, markerfacecolor=color))

        center = np.mean(np.array(rect), axis=0)
        centers.append(center)
        ax.scatter(*center, color='red', s=30)
        ax.text(center[0]-5, center[1]-5, center[2], f"P{i}", color=color, alpha = 0.8, fontsize=12)

    # Linie durch die berechneten Mittelpunkte
    centers = np.array(centers)
    ax.plot(centers[:, 0], centers[:, 1], centers[:, 2], color='black', linewidth=2)

    # Beschriftung Hauptlinie
    if len(centers) > 0:
        ax.text(*centers[0], 'Pre-Position', color='black', fontsize=10)
    if len(centers) > 2:
        ax.text(*centers[-1], 'Loslassen', color='black', fontsize=10)

    # Optimal-Trajektorie (grün)
    if secondary_rectangles is not None:
        secondary_centers = []
        for rect in secondary_rectangles:
            poly = Poly3DCollection([rect], facecolors='green', alpha=0.1, edgecolors='green')
            ax.add_collection3d(poly)

            center = np.mean(np.array(rect), axis=0)
            secondary_centers.append(center)
            ax.scatter(*center, color='green', s=20)

        # Linie durch grüne Mittelpunkte
        secondary_centers = np.array(secondary_centers)
        ax.plot(secondary_centers[:, 0], secondary_centers[:, 1], secondary_centers[:, 2],
                color='green', linewidth=1, linestyle='-')

        # Legendenhandle für Optimal-Trajektorie
        if secondary_legend_title:
            legend_handles.append(plt.Line2D([0], [0], marker='o', color='w', label=secondary_legend_title,
                                             markersize=10, markerfacecolor='green'))

    # Einzeichnen von Spitze und Senke inkl. graue Verbindungslinie
    if connect_points is not None and len(connect_points) == 2:
        p1, p2 = np.array(connect_points[0]), np.array(connect_points[1])
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]],
                color='gray', linestyle='dashed', linewidth=2)
        ax.scatter(*p1, 'o', color='magenta', linewidth=6)
        ax.scatter(*p2, 'o', color='magenta', linewidth=6)
        ax.text(*p1, 'Spitze', color='magenta', fontsize=12, ha='center')
        ax.text(*p2, 'Senke', color='magenta', fontsize=12, ha='center')

    # Achseneinstellungen
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f"Haken {hook_num}, Trajektorien-Ansatz {trajectory_process}", fontsize = 10)

    all_points = rectangles + (secondary_rectangles if secondary_rectangles else [])
    all_points = np.vstack(all_points)
    mid = np.mean(all_points, axis=0)
    max_range = np.ptp(all_points, axis=0).max() / 2
    for i, axis in enumerate([ax.set_xlim, ax.set_ylim, ax.set_zlim]):
        axis(mid[i] - max_range, mid[i] + max_range)

    ax.set_box_aspect([1, 1, 1])
    ax.legend(handles=legend_handles, title="Trajektorien-Punkte", loc='upper left')
    plt.tight_layout(pad=0.5, w_pad=0.5, h_pad=0.5)
    plt.show()
'''

'\ndef visualize_rectangles(rectangles, figsize=(8, 8), connect_points=None, \n                         secondary_rectangles=None, secondary_legend_title=None,\n                         hook_num = 0, trajectory_process = 1):\n    """\n    Visualisiert Rechtecke + Trajektorie im 3D-Raum mit Legende und Bezeichnungen der Trajektorienpunkte.\n    \n    Zusätzlich kann ein zweiter Satz Rechtecke (grau & transparent) dargestellt werden – inkl. Mittelpunktslinie.\n\n    :param rectangles: Liste von Rechtecken (jeweils 4 Punkte [x, y, z])\n    :param figsize: Größe der Plotfläche (Breite, Höhe)\n    :param connect_points: Optionales Tupel mit zwei Punkten [x, y, z], die verbunden werden sollen\n    :param secondary_rectangles: Optionaler zweiter Satz Rechtecke (werden grau dargestellt)\n    :param secondary_legend_title: Optionaler Legenden-Eintrag für die grauen Rechtecke\n    """\n    %matplotlib qt\n\n    fig = plt.figure(figsize=figsize)\n    ax = fig.add_subplot(111, projection=\'3d\')\n

In [19]:
'''
visualize_rectangles(rectangles = rectangles_1,
                     secondary_rectangles = rectangles_optim,
                     secondary_legend_title = 'optimale Trajektorie',
                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
                     hook_num = hook_num,
                     trajectory_process = 1)

visualize_rectangles(rectangles = rectangles_2,
                     secondary_rectangles = rectangles_optim,
                     secondary_legend_title = 'optimale Trajektorie',
                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
                     hook_num = hook_num,
                     trajectory_process = 2)

visualize_rectangles(rectangles = rectangles_3,
                     secondary_rectangles = rectangles_optim,
                     secondary_legend_title = 'optimale Trajektorie',
                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
                     hook_num = hook_num,
                     trajectory_process = 3)

visualize_rectangles(rectangles = rectangles_4,
                     secondary_rectangles = rectangles_optim,
                     secondary_legend_title = 'optimale Trajektorie',
                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
                     hook_num = hook_num,
                     trajectory_process = 4)
'''

"\nvisualize_rectangles(rectangles = rectangles_1,\n                     secondary_rectangles = rectangles_optim,\n                     secondary_legend_title = 'optimale Trajektorie',\n                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),\n                     hook_num = hook_num,\n                     trajectory_process = 1)\n\nvisualize_rectangles(rectangles = rectangles_2,\n                     secondary_rectangles = rectangles_optim,\n                     secondary_legend_title = 'optimale Trajektorie',\n                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),\n                     hook_num = hook_num,\n                     trajectory_process = 2)\n\nvisualize_rectangles(rectangles = rectangles_3,\n                     secondary_rectangles = rectangles_optim,\n                     secondary_legend_title = 'optimale Trajektorie',\n                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),\n                    

In [20]:
def visualize_rectangles_html(rectangles, connect_points=None, 
                              secondary_rectangles=None, secondary_legend_title=None,
                              hook_num=0, trajectory_process=1, html_filename="/home/mo/Thesis/Evaluation/Trajektorientests/CSV/plot_interaktiv.html"):
    """
    Visualisiert Rechtecke und Trajektorien im 3D-Raum als interaktive HTML-Datei.
    
    :param rectangles: Liste von Rechtecken (jeweils 4 Punkte [x, y, z])
    :param figsize: Größe der Plotfläche (Breite, Höhe)
    :param connect_points: Optionales Tupel mit zwei Punkten [x, y, z], die verbunden werden sollen
    :param secondary_rectangles: Optionaler zweiter Satz Rechtecke (werden grau dargestellt)
    :param secondary_legend_title: Optionaler Legenden-Eintrag für die grauen Rechtecke
    :param html_filename: Der Name der HTML-Datei, die erstellt wird
    """
    fig = go.Figure()

    # Berechnete Trajektorie (farbig)
    cmap = plt.cm.get_cmap('tab20')
    centers = []

    if rectangles:
        for i, rect in enumerate(rectangles):
            # Verwende rgba-String für die Farbe (anstatt des RGBA-Tupels)
            '''
            color = f"rgba({int(cmap(i / len(rectangles))[0] * 255)}, " \
                    f"{int(cmap(i / len(rectangles))[1] * 255)}, " \
                    f"{int(cmap(i / len(rectangles))[2] * 255)}, 1.0)"
            '''
            color = 'red'
            x, y, z = zip(*rect)

            fig.add_trace(go.Mesh3d(x=x, y=y, z=z, color=color, opacity=0.1, 
                                   name=f'P{i+1}'))
        

            center = np.mean(np.array(rect), axis=0)
            centers.append(center)

            '''
            # Text für "P0", "P1", etc.
            if i == 0:
                text_label = ''
                # text_label = 'Pre-Position'
            elif i == 1:
                text_label = ''
                # text_label = 'Spitze'
            elif i == len(rectangles) - 1:
                text_label = ''
                # text_label = 'Loslassen'
            else:
                text_label = f'P{i}'  # Standardtext für den Punkt
            '''
            text_label = ''
            # Füge den Text an den Punkten hinzu (mit entsprechender Position)
            fig.add_trace(go.Scatter3d(x=[center[0]], y=[center[1]], z=[center[2]], 
                                       mode='markers+text', marker=dict(size=3, color='red'),
                                       text=text_label, textposition="top center", textfont=dict(size=12), showlegend=False))

            
            # Hinzufügen der Umrandung der Rechtecke mit einer Linie
            if i == 0 or i == (len(rectangles) - 1 ):
                for j in range(4):
                    x0, y0, z0 = rect[j]
                    x1, y1, z1 = rect[(j + 1) % 4]  # Nächster Punkt im Rechteck
                    fig.add_trace(go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1], 
                                               mode='lines', 
                                               line=dict(color=color, width=4),  # Keine opacity hier
                                               showlegend=False))
            

        # Linie durch die berechneten Mittelpunkte
        centers = np.array(centers)
        fig.add_trace(go.Scatter3d(x=centers[:, 0], y=centers[:, 1], z=centers[:, 2],
                                   mode='lines', line=dict(color='black', width=2), name='Berechnete Hakenlinie'))

        # Beschriftung der ersten und letzten Punkte
        if len(centers) > 0:
            fig.add_trace(go.Scatter3d(x=[centers[0, 0]], y=[centers[0, 1]], z=[centers[0, 2]],
                                       mode='text', text=['Pre-Position'], textposition="top center", textfont=dict(size=10), showlegend=False))
        if len(centers) > 2:
            fig.add_trace(go.Scatter3d(x=[centers[-1, 0]], y=[centers[-1, 1]], z=[centers[-1, 2]],
                                       mode='text', text=['Loslassen'], textposition="top center", textfont=dict(size=10), showlegend=False))
        
        # Tunnelkanten: Ecken durchziehen mit denselben Farben wie bei den Rechtecken
        num_corners = 4  # Immer 4 Ecken pro Rechteck
        for corner_idx in range(num_corners):
            x_line, y_line, z_line = [], [], []
            colors = []
            for i, rect in enumerate(rectangles):
                # Farbcode für das Rechteck
                colors.append(f"rgba({int(cmap(i / len(rectangles))[0] * 255)}, " \
                        f"{int(cmap(i / len(rectangles))[1] * 255)}, " \
                        f"{int(cmap(i / len(rectangles))[2] * 255)}, 1.0)")
                            
                x, y, z = rect[corner_idx]
                x_line.append(x)
                y_line.append(y)
                z_line.append(z)
            
            # Die Tunnelkanten mit dem Farbcode der Rechtecke zeichnen
            fig.add_trace(go.Scatter3d(x=x_line, y=y_line, z=z_line,
                                       mode='lines',
                                       line=dict(color='red', width=3),
                                       showlegend=False))

    # Optimal-Trajektorie (grün) – Zweite Rechtecke
    if secondary_rectangles:
        for i,rect in enumerate(secondary_rectangles):
            x, y, z = zip(*rect)
            
            fig.add_trace(go.Mesh3d(x=x, y=y, z=z, color='rgba(0,255,0,1.0)', opacity=0.1, 
                                   name=secondary_legend_title or 'Optimale Trajektorie'))
            
            
            # Hinzufügen der Umrandung der Rechtecke mit einer Linie
            if i == 0 or i == (len(rectangles) - 1):
                for j in range(4):
                    x0, y0, z0 = rect[j]
                    x1, y1, z1 = rect[(j + 1) % 4]  # Nächster Punkt im Rechteck
                    fig.add_trace(go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1], 
                                               mode='lines', 
                                               line=dict(color='rgba(0,255,0,1.0)', width=3),  # Keine opacity hier
                                               showlegend=False))
            
            
            # Hinzufügen eines roten Punktes in der Mitte des Rechtecks
            center = np.mean(np.array(rect), axis=0)
            fig.add_trace(go.Scatter3d(x=[center[0]], y=[center[1]], z=[center[2]], 
                                       mode='markers', marker=dict(size=2, color='green'), 
                                       showlegend=False))
        
        # Mittelpunkte der sekundären Rechtecke sammeln
        secondary_centers = [np.mean(np.array(rect), axis=0) for rect in secondary_rectangles]
        secondary_centers = np.array(secondary_centers)

        # Linie durch die Mittelpunkte zeichnen
        fig.add_trace(go.Scatter3d(
            x=secondary_centers[:, 0], y=secondary_centers[:, 1], z=secondary_centers[:, 2],
            mode='lines',
            line=dict(color='rgba(0,255,0,1.0)', width=2),
            name='Optimale Hakenlinie'
        ))
    
    # Einzeichnen von Spitze und Senke inkl. graue Verbindungslinie
    if connect_points:
        p1, p2 = np.array(connect_points[0]), np.array(connect_points[1])
        fig.add_trace(go.Scatter3d(x=[p1[0], p2[0]], y=[p1[1], p2[1]], z=[p1[2], p2[2]], 
                                   mode='lines', line=dict(color='gray', dash='dash'), name='Direkte Gerade Spitze -> Senke'))
        fig.add_trace(go.Scatter3d(x=[p1[0]], y=[p1[1]], z=[p1[2]], mode='markers', 
                                   marker=dict(size=8, color='magenta'), name='Spitze'))
        fig.add_trace(go.Scatter3d(x=[p2[0]], y=[p2[1]], z=[p2[2]], mode='markers', 
                                   marker=dict(size=8, color='magenta'), name='Senke'))
        
        # Hinzufügen von Text "Spitze" und "Senke"
        fig.add_trace(go.Scatter3d(x=[p1[0]], y=[p1[1]], z=[p1[2]], mode='text', 
                                   text=['Spitze'], textposition="top center", textfont=dict(size=10), showlegend=False))
        fig.add_trace(go.Scatter3d(x=[p2[0]], y=[p2[1]], z=[p2[2]], mode='text', 
                                   text=['Senke'], textposition="top center", textfont=dict(size=10), showlegend=False))
        
    # Tunnelkanten Optimal-Trajektorie
    num_corners = 4  # Immer 4 Ecken pro Rechteck
    for corner_idx in range(num_corners):
        x_line, y_line, z_line = [], [], []
        colors = []
        for i, rect in enumerate(secondary_rectangles):                       
            x, y, z = rect[corner_idx]
            x_line.append(x)
            y_line.append(y)
            z_line.append(z)
        
        # Die Tunnelkanten mit dem Farbcode der Rechtecke zeichnen
        fig.add_trace(go.Scatter3d(x=x_line, y=y_line, z=z_line,
                                   mode='lines',
                                   line=dict(color='rgba(0,255,0,1.0)', width=3),
                                   showlegend=False))
        
    # Achsen und Layout
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube',
            camera=dict(
                up=dict(x=0, y=-1., z=0),
                eye=dict(x=-1, y=-1, z=-1.5)
            )
        ),
        title=f'Haken {hook_num}, Trajektorien-Ansatz {trajectory_process}',
        legend=dict(title="Legende")
    )
    
    # HTML exportieren
    fig.write_html(html_filename)
    print(f"HTML-Datei '{html_filename}' erfolgreich erstellt!")
    fig.show()

In [21]:
visualize_rectangles_html(
    rectangles = rectangles_1,
    secondary_rectangles = rectangles_optim,
    secondary_legend_title = 'optimale Trajektorie',
    connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
    hook_num = hook_num,
    trajectory_process = 1,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj1.html'
    )

visualize_rectangles_html(
    rectangles = rectangles_2,
    secondary_rectangles = rectangles_optim,
    secondary_legend_title = 'optimale Trajektorie',
    connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
    hook_num = hook_num,
    trajectory_process = 2,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj2.html'
    )

visualize_rectangles_html(
    rectangles = rectangles_3,
    secondary_rectangles = rectangles_optim,
    secondary_legend_title = 'optimale Trajektorie',
    connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
    hook_num = hook_num,
    trajectory_process = 3,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj3.html'
    )

visualize_rectangles_html(
    rectangles = rectangles_4,
    secondary_rectangles = rectangles_optim,
    secondary_legend_title = 'optimale Trajektorie',
    connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
    hook_num = hook_num,
    trajectory_process = 4,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj4.html'
    )

/tmp/ipykernel_1316384/1159511825.py:17: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj1.html' erfolgreich erstellt!


HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj2.html' erfolgreich erstellt!


HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj3.html' erfolgreich erstellt!


HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj4.html' erfolgreich erstellt!
